In [130]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch.exceptions import NotFoundError
import os
import numpy as np
from tqdm import tqdm
import sys
from time import sleep
import re
from morfeusz2 import Morfeusz
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
from collections import Counter
import pylab
import matplotlib.pyplot as plt
import random
import Levenshtein
import functools
import time

### Załadowanie datasetów

In [131]:
from datasets import load_dataset

# ograniczenie liczby wpisów, bo na pełnym datasecie nie jestem w stanie policzyć liczby wystąpień tokenów
fiqa_pl = load_dataset("clarin-knext/fiqa-pl", 'corpus', split='corpus[:100%]')
fiqa_pl


Dataset({
    features: ['_id', 'title', 'text'],
    num_rows: 57638
})

In [132]:
from datasets import load_dataset

fiqa_queries_pl = load_dataset("clarin-knext/fiqa-pl", 'queries', split='queries[:100%]')
fiqa_queries_pl


Dataset({
    features: ['_id', 'title', 'text'],
    num_rows: 6648
})

### Tokenizacja każdego wpisu w datasecie osobno

In [133]:
nlp = Polish()
tokenizer = nlp.tokenizer

In [134]:
tokens_all = []

for entry in tqdm(fiqa_pl):
    text = entry['text']
    tokens = [token for token in tokenizer(text)]
    tokens_all.append(tokens)

tokens_all

100%|██████████| 57638/57638 [00:40<00:00, 1422.02it/s]


[[Nie,
  mówię,
  ,,
  że,
  nie,
  podoba,
  mi,
  się,
  też,
  pomysł,
  szkolenia,
  w,
  miejscu,
  pracy,
  ,,
  ale,
  nie,
  możesz,
  oczekiwać,
  ,,
  że,
  firma,
  to,
  zrobi,
  .,
  Szkolenie,
  pracowników,
  to,
  nie,
  ich,
  praca,
  –,
  oni,
  tworzą,
  oprogramowanie,
  .,
  Być,
  może,
  systemy,
  edukacyjne,
  w,
  Stanach,
  Zjednoczonych,
  (,
  lub,
  ich,
  studenci,
  ),
  powinny,
  trochę,
  martwić,
  się,
  o,
  zdobycie,
  umiejętności,
  rynkowych,
  w,
  zamian,
  za,
  ich,
  ogromne,
  inwestycje,
  w,
  edukację,
  ,,
  zamiast,
  wychodzić,
  z,
  tysiącami,
  zadłużonych,
  studentów,
  i,
  narzekać,
  ,,
  że,
  nie,
  są,
  do,
  niczego,
  wykwalifikowani,
  .],
 [Tak,
  więc,
  nic,
  nie,
  zapobiega,
  fałszywym,
  ocenom,
  poza,
  dodatkową,
  kontrolą,
  ze,
  strony,
  rynku,
  /,
  inwestorów,
  ,,
  ale,
  istnieją,
  pewne,
  nowsze,
  kontrole,
  ,,
  które,
  uniemożliwiają,
  instytucjom,
  korzystanie,
  z,
  nich,
  .,
  W,


In [135]:
q_tokens_all = dict()

for entry in tqdm(fiqa_queries_pl):
    text = entry['text']
    tokens = [token for token in tokenizer(text)]
    q_tokens_all[entry['_id']] = tokens

q_tokens_all

100%|██████████| 6648/6648 [00:00<00:00, 10798.85it/s]


{'0': [Co, jest, uważane, za, wydatek, służbowy, w, podróży, służbowej, ?],
 '4': [Wydatki,
  służbowe,
  -,
  ubezpieczenie,
  samochodu,
  podlegające,
  odliczeniu,
  za,
  wypadek,
  ,,
  który,
  wydarzył,
  się,
  podczas,
  podróży,
  służbowej],
 '5': [Rozpoczęcie, nowego, biznesu, online],
 '6': [„, Dzień, roboczy, ”, i, „, termin, płatności, ”, rachunków],
 '7': [Nowy,
  właściciel,
  firmy,
  –,
  Jak,
  działają,
  podatki,
  dla,
  firmy,
  i,
  osoby,
  fizycznej,
  ?],
 '9': [Hobby, kontra, biznes],
 '11': [Czeki, osobiste, zamiast, firmowych],
 '12': [Czy,
  amerykański,
  kodeks,
  podatkowy,
  wymaga,
  ,,
  aby,
  właściciele,
  małych,
  firm,
  liczyli,
  zakupy,
  biznesowe,
  jako,
  dochód,
  osobisty,
  ?],
 '13': [Jak,
  mogę,
  zarejestrować,
  firmę,
  w,
  Wielkiej,
  Brytanii,
  bez,
  podawania,
  adresu,
  firmy,
  ?],
 '14': [Czym, są, „, podstawy, biznesowe, ”, ?],
 '16': [Strata, na, inwestycjach, biznesowych, z, poprzedniego, roku],
 '19': [Jak,
  mo

In [136]:
import pickle

# with open('fiqa_pl_tokens.pickle', 'rb') as f:
#     tokens_all = pickle.load(f)

# with open('fiqa_pl_queries_tokens.pickle', 'rb') as f:
#     q_tokens_all = pickle.load(f)



# with open('fiqa_pl_tokens.pickle', 'wb') as f:
#     pickle.dump(tokens_all, f)

# with open('fiqa_pl_queries_tokens.pickle', 'wb') as f:
#     pickle.dump(q_tokens_all, f)



In [175]:
counts_all = [Counter([a.text for a in t]) for t in tqdm(tokens_all)]
q_counts_all = [Counter([a.text for a in t]) for t in tqdm(q_tokens_all.values())]

100%|██████████| 6648/6648 [00:00<00:00, 143511.62it/s]


In [ ]:
fiqa_pl = fiqa_pl.add_column('tokens_counts', counts_all)
fiqa_pl = fiqa_pl.add_column('tokens', tokens_all)

In [45]:
# with open('fiqa_pl_token_counts.pickle', 'wb') as f:
    # pickle.dump(counts_all, f)
# with open('fiqa_pl_queries_token_counts.pickle', 'wb') as f:
#     pickle.dump(q_counts_all, f)


### Zliczenie globalnej liczby wystąpień tokenów

In [176]:
global_token_counts = Counter([])

for counter in counts_all:
    global_token_counts.update(counter)

for counter in q_counts_all:
    global_token_counts.update(counter)

global_token_counts

Counter({',': 613534,
         '.': 404067,
         'w': 160393,
         'i': 124592,
         'na': 115887,
         'nie': 113095,
         'to': 100826,
         'z': 94443,
         'że': 90074,
         'jest': 89793,
         'się': 86370,
         'do': 66473,
         ')': 57383,
         '(': 48403,
         '„': 41711,
         '”': 37656,
         'o': 37584,
         'a': 36999,
         '-': 36810,
         'są': 34494,
         'ale': 34311,
         'lub': 33029,
         '?': 32190,
         'za': 32047,
         '*': 31507,
         'jak': 30462,
         'od': 29442,
         'aby': 29246,
         '"': 28968,
         '/': 28536,
         'jeśli': 28293,
         'co': 27018,
         'dla': 24642,
         'może': 24118,
         'które': 24044,
         'Jeśli': 23932,
         ':': 22848,
         'tego': 21853,
         'po': 21580,
         'tym': 21569,
         '%': 20922,
         'ma': 19847,
         'tak': 19466,
         'możesz': 18652,
         'być':

### Zaaplikowanie losowego psucia słów w datasecie 'queries'

In [147]:
import string
from copy import deepcopy

q_broken_tokens = dict()
change = dict()

for _id, tokens in q_tokens_all.items():
    i = random.randrange(len(tokens))
    token = tokens[i].text
    j = random.randrange(len(token))

    broken = token[:j] + random.choice(string.ascii_letters) + token[j+1:]
    
    change[_id] = (i, tokens[i].idx, broken)
    q_broken_tokens[_id] = []
    for l in range(len(tokens)):
        q_broken_tokens[_id].append(tokens[l].text)
    q_broken_tokens[_id][i] = broken

q_broken_tokens

{'0': ['Co',
  'jest',
  'uważane',
  'za',
  'wydatek',
  'słZżbowy',
  'w',
  'podróży',
  'służbowej',
  '?'],
 '4': ['Wydatki',
  'służbowe',
  '-',
  'ubezpieczenie',
  'samochodu',
  'podlegające',
  'odliczeniu',
  'za',
  'wypadek',
  ',',
  'któro',
  'wydarzył',
  'się',
  'podczas',
  'podróży',
  'służbowej'],
 '5': ['Rozpoczęcie', 'nowego', 'biznesu', 'onlint'],
 '6': ['„',
  'Dzień',
  'roboczy',
  '”',
  'i',
  '„',
  'termin',
  'pzatności',
  '”',
  'rachunków'],
 '7': ['Nowy',
  'właściciel',
  'firmy',
  '–',
  'Jak',
  'działają',
  'podatki',
  'dla',
  'firmy',
  'i',
  'osoby',
  'fizyjznej',
  '?'],
 '9': ['Hobby', 'konQra', 'biznes'],
 '11': ['Czeki', 'osobiste', 'zamiast', 'firmowyGh'],
 '12': ['Czy',
  'amerykański',
  'kodeks',
  'podatkowy',
  'wymaga',
  ',',
  'aby',
  'właściciele',
  'małych',
  'firm',
  'liczyli',
  'zakupy',
  'biznesowe',
  'jako',
  'dochód',
  'osobisty',
  'i'],
 '13': ['Jak',
  'mogę',
  'zarejestAować',
  'firmę',
  'w',
  'Wie

In [148]:
import pandas as pd
from datasets import load_dataset

fiqa_pl_qrels = load_dataset("clarin-knext/fiqa-pl-qrels", split='test').to_pandas()
fiqa_pl_pd = load_dataset("clarin-knext/fiqa-pl", 'corpus', split='corpus').to_pandas()
questions = load_dataset("clarin-knext/fiqa-pl", 'queries', split='queries').to_pandas()
questions['_id'] = questions['_id'].astype(int)
fiqa_pl_pd['_id'] = fiqa_pl_pd['_id'].astype(int)
fiqa_pl_qrels['query-id'] = fiqa_pl_qrels['query-id'].astype(int)
fiqa_pl_qrels['corpus-id'] = fiqa_pl_qrels['corpus-id'].astype(int)

qa_intermediate = pd.merge(questions, fiqa_pl_qrels, left_on='_id', right_on='query-id')
qa_pairs = pd.merge(qa_intermediate, fiqa_pl_pd, left_on='corpus-id', right_on='_id', suffixes=('_question', '_answer'))
qa_pairs = qa_pairs[['query-id', 'text_question', 'corpus-id', 'score', 'text_answer']]
qa_pairs

,query-id,text_question,corpus-id,score,text_answer
0,4641,Gdzie powinienem zaparkować mój fundusz na des...,44594,1,"„Po pierwsze, zazwyczaj chcesz zaparkować swój..."
1,4641,Gdzie powinienem zaparkować mój fundusz na des...,406219,1,Sugerowałbym lokalną kasę kredytową lub lokaln...
2,4641,Gdzie powinienem zaparkować mój fundusz na des...,319954,1,"Używam ING do awaryjnych oszczędności, ale w z..."
3,4641,Gdzie powinienem zaparkować mój fundusz na des...,397358,1,"To chyba dobry moment, aby zauważyć, że kredyt..."
4,4641,Gdzie powinienem zaparkować mój fundusz na des...,88327,1,"Coś z gwarancją FDIC, czyli bank. W przypadku ..."
...,...,...,...,...,...
1701,94,Wykorzystywanie punktów kart kredytowych do op...,245447,1,„Dla uproszczenia zacznijmy od rozważenia zwro...
1702,2551,Jak znaleźć tańszą alternatywę dla tradycyjnej...,413832,1,Najtańszy to jedno. Możesz absolutnie robić za...
1703,2551,Jak znaleźć tańszą alternatywę dla tradycyjnej...,450742,1,"„To, jak niskie możesz obniżyć koszty, zależy ..."
1704,2551,Jak znaleźć tańszą alternatywę dla tradycyjnej...,143100,1,Spróbuj użyć dostawcy usług VOIP lub interneto...


In [149]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import RequestError

es = Elasticsearch("http://localhost:9200")

index_name = "fiqa_pl_corpus"

def query_es(question, index_name, field_name, k):
    query = {
        "query": {
            "match": {
                field_name: question
            }
        },
        "size": k
    }
    response = es.search(index=index_name, body=query)
    return response["hits"]["hits"]

def get_dcg(rel, k):
    pos = [i for i in range(1, k + 1)]
    discounting_factor = [1/np.log(1 + i) for i in range(1, k + 1)]
    dcg = sum([rel[i] * discounting_factor[i] for i in range(k)])
    return dcg

def get_ndcg(rel, k):
    dcg = get_dcg(rel, k)    
    ideal_rel = sorted(rel, reverse=True)
    idcg = get_dcg(ideal_rel, k)
    ndcg = dcg / idcg
    return ndcg

configs = [
    "content_with_synonyms_lema",
    "content_with_synonyms_no_lema",
    "content_without_synonyms_lema",
    "content_without_synonyms_no_lema",
]

ndcg_scores = { config.removeprefix('content_'): [] for config in configs }

for i, qa_pair in enumerate(qa_pairs.iterrows()):
    qa_pair = qa_pair[1]
    question = qa_pair.iloc[1]
    q_id = qa_pair['query-id']
    c = change[str(q_id)]
    broken_token = c[2]
    question = question[:c[1]] + broken_token + question[c[1] + len(broken_token):]
    
    for field_name in configs:
        retrieved_results = query_es(question, "fiqa_pl_corpus", field_name, k=10)
        # print(retrieved_results)
        
        retrieved_relevance = [result["_score"] for result in retrieved_results]
        score = get_ndcg(retrieved_relevance, k=min(10, len(retrieved_relevance)))
        key = field_name.removeprefix('content_')
        ndcg_scores[key].append(score)

# Calculate average NDCG@5 for each configuration
avg_ndcg_scores = {config: np.mean(scores) for config, scores in ndcg_scores.items()}
print("Average NDCG@5 for each configuration:", avg_ndcg_scores)


Average NDCG@5 for each configuration: {'with_synonyms_lema': np.float64(1.0), 'with_synonyms_no_lema': np.float64(1.0), 'without_synonyms_lema': np.float64(1.0), 'without_synonyms_no_lema': np.float64(1.0)}


### Analiza zepsutych query morfeuszem

In [150]:
import morfeusz2

morf = morfeusz2.Morfeusz()

def word_in_dictionary(word):
    analysis = morf.analyse(word)
    if len(analysis) == 0:
        return False
    return analysis[0][2][2] != 'ign'


In [152]:
flat_broken_tokens = [t for tt in q_broken_tokens.values() for t in tt]

words_not_in_morf = list(filter(lambda w: not word_in_dictionary(w[0]), flat_broken_tokens))
words_in_morf  = list(filter(lambda w: word_in_dictionary(w[0]), flat_broken_tokens))
words_not_in_morf

['Rozpoczęcie',
 'nowego',
 'roboczy',
 'rachunków',
 'Nowy',
 'firmy',
 'firmy',
 'fizyjznej',
 'firmowyGh',
 'firm',
 'firmę',
 'firmy',
 'na',
 'roku',
 'rogę',
 'firmy',
 'firmy',
 'nieudokumentowanych',
 'firmy',
 'rozpofzynanie',
 'na',
 'firmy',
 'Rozpoczęcie',
 'niezbyt',
 'rozliczania',
 'firmy',
 'firmy',
 'firmowego',
 'nie',
 'realizować',
 'na',
 'firmy',
 'Najlepsza',
 'NowB',
 'nie',
 'firmy',
 'firmę',
 'firmy',
 'rodzinnej',
 'ramach',
 'formie',
 'firmy',
 'raportowanie',
 'niezrefundowanXch',
 'firm',
 'firmowego',
 'nie',
 'na',
 'firma',
 'firmowe',
 'rozliczeniowe',
 'rejestrować',
 'firmy',
 'na',
 'firmę',
 'równy',
 'niezapłacone',
 'na',
 'nagród',
 'firmowej',
 'firmę',
 'firmy',
 'Na',
 'firmowe',
 'rw',
 'fWrmule',
 'firmy',
 'Roth',
 'rozpocząć',
 'Naprawdę',
 'firmowej',
 'firma',
 'firmy',
 'firmy',
 'ne',
 'firmy',
 'żony',
 'na',
 'nagród',
 'na',
 'F',
 'natychmiastowe',
 'firmy',
 'naliczają',
 'nawzajem',
 'Formularz',
 'formularz',
 'firma',
 'niż'

In [154]:
words_in_morf

['Co',
 'jest',
 'uważane',
 'za',
 'wydatek',
 'słZżbowy',
 'w',
 'podróży',
 'służbowej',
 '?',
 'Wydatki',
 'służbowe',
 '-',
 'ubezpieczenie',
 'samochodu',
 'podlegające',
 'odliczeniu',
 'za',
 'wypadek',
 ',',
 'któro',
 'wydarzył',
 'się',
 'podczas',
 'podróży',
 'służbowej',
 'biznesu',
 'onlint',
 '„',
 'Dzień',
 '”',
 'i',
 '„',
 'termin',
 'pzatności',
 '”',
 'właściciel',
 '–',
 'Jak',
 'działają',
 'podatki',
 'dla',
 'i',
 'osoby',
 '?',
 'Hobby',
 'konQra',
 'biznes',
 'Czeki',
 'osobiste',
 'zamiast',
 'Czy',
 'amerykański',
 'kodeks',
 'podatkowy',
 'wymaga',
 ',',
 'aby',
 'właściciele',
 'małych',
 'liczyli',
 'zakupy',
 'biznesowe',
 'jako',
 'dochód',
 'osobisty',
 'i',
 'Jak',
 'mogę',
 'zarejestAować',
 'w',
 'Wielkiej',
 'Brytanii',
 'bez',
 'podawania',
 'adresu',
 '?',
 'Czym',
 'ss',
 '„',
 'podstawy',
 'biznesowe',
 '”',
 '?',
 'dtrata',
 'inwestycjach',
 'biznesowych',
 'z',
 'poprzedniego',
 'Jak',
 'oszacować',
 'podatki',
 'biznesowe',
 '/',
 'opłaty',

In [179]:
import morfeusz2
from collections import Counter
import itertools


words_to_correct = words_not_in_morf

def generate_candidates(word):
    letters = 'abcdefghijklmnopqrstuvwxyząćęłńóśźż'
    letters += letters.upper()
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [
        L + R[1] + R[0] + R[2:] 
        for L, R in splits 
        if len(R) > 1
        ]
    replaces = [
        L + c + R[1:] 
        for L, R in splits
        if R for c in letters
        ]
    inserts = [
        L + c + R 
        for L, R in splits for c in letters
        ]
    
    return set(deletes + transposes + replaces + inserts)

def valid_candidate(candidate, frequency_list):
    return frequency_list.get(candidate, 0)

def correct_word(word, frequency_list):
    candidates = generate_candidates(word)
    valid_candidates = [
        (candidate, valid_candidate(candidate, frequency_list)) 
        for candidate in candidates 
        if candidate in frequency_list
        ]


    if valid_candidates:
        return max(valid_candidates, key=lambda x: x[1])[0]
    else:
        return word

corrected_words = dict()
for word_to_correct in words_to_correct:
    corrected_word = correct_word(word_to_correct, global_token_counts)
    if corrected_word != word_to_correct:
        corrected_words[word_to_correct] = corrected_word

corrected_words

{'Rozpoczęcie': 'rozpoczęcie',
 'roboczy': 'roboczą',
 'Nowy': 'nowy',
 'fizyjznej': 'fizycznej',
 'firmowyGh': 'firmowych',
 'firm': 'firmy',
 'firmę': 'firmy',
 'rogę': 'mogę',
 'rozpofzynanie': 'rozpoczynanie',
 'rozliczania': 'rozliczenia',
 'realizować': 'zrealizować',
 'Najlepsza': 'Najlepsze',
 'NowB': 'Nowy',
 'rodzinnej': 'rodzinne',
 'formie': 'firmie',
 'raportowanie': 'raportowania',
 'niezrefundowanXch': 'niezrefundowanych',
 'firma': 'firmy',
 'rozliczeniowe': 'rozliczeniowy',
 'równy': 'równe',
 'niezapłacone': 'niezapłacony',
 'firmowej': 'firmowe',
 'Na': 'na',
 'rw': 'w',
 'fWrmule': 'formule',
 'Naprawdę': 'naprawdę',
 'ne': 'na',
 'żony': 'żona',
 'F': 'w',
 'Formularz': 'formularz',
 'niż': 'nie',
 'robić': 'zrobić',
 'finansująca': 'finansując',
 'numeru': 'numer',
 'fipma': 'firma',
 'fuszę': 'muszę',
 'finansowo': 'finansowe',
 'nim': 'nie',
 'ryzyka': 'ryzyko',
 'nazwę': 'nazwy',
 'Naliczanie': 'naliczanie',
 'Niespodzianka': 'niespodzianka',
 'rodzaj': 'rodzaj

In [180]:

ndcg_scores = { config.removeprefix('content_'): [] for config in configs }

for i, qa_pair in enumerate(qa_pairs.iterrows()):
    qa_pair = qa_pair[1]
    question = qa_pair.iloc[1]
    q_id = qa_pair['query-id']

    # break the query by introducing typo
    c = change[str(q_id)]
    broken_token = c[2]
    question = question[:c[1]] + broken_token + question[c[1] + len(broken_token):]

    # repair question again
    question.replace(broken_token, corrected_words.get(broken_token, broken_token))
    
    for field_name in configs:
        retrieved_results = query_es(question, "fiqa_pl_corpus", field_name, k=10)
        # print(retrieved_results)
        
        retrieved_relevance = [result["_score"] for result in retrieved_results]
        score = get_ndcg(retrieved_relevance, k=min(10, len(retrieved_relevance)))
        key = field_name.removeprefix('content_')
        ndcg_scores[key].append(score)

# Calculate average NDCG@5 for each configuration
avg_ndcg_scores = {config: np.mean(scores) for config, scores in ndcg_scores.items()}
print("Average NDCG@5 for each configuration:", avg_ndcg_scores)


Average NDCG@5 for each configuration: {'with_synonyms_lema': np.float64(1.0), 'with_synonyms_no_lema': np.float64(1.0), 'without_synonyms_lema': np.float64(1.0), 'without_synonyms_no_lema': np.float64(1.0)}


In [181]:
def fuzzy_query_es(question, index_name, field_name, k):
    query = {
        "query": {
            "fuzzy": {
                field_name: {
                    'value': question,
                    'fuzziness': 1
                }
            }
        },
        "size": k
    }
    response = es.search(index=index_name, body=query)
    return response["hits"]["hits"]


ndcg_scores = { config.removeprefix('content_'): [] for config in configs }

for i, qa_pair in enumerate(qa_pairs.iterrows()):
    qa_pair = qa_pair[1]
    question = qa_pair.iloc[1]
    q_id = qa_pair['query-id']

    # break the query by introducing typo
    c = change[str(q_id)]
    broken_token = c[2]
    question = question[:c[1]] + broken_token + question[c[1] + len(broken_token):]


    for field_name in configs:
        retrieved_results = fuzzy_query_es(question, "fiqa_pl_corpus", field_name, k=10)
        # print(retrieved_results)
        
        retrieved_relevance = [result["_score"] for result in retrieved_results]
        score = get_ndcg(retrieved_relevance, k=min(10, len(retrieved_relevance)))
        key = field_name.removeprefix('content_')
        ndcg_scores[key].append(score)

# Calculate average NDCG@5 for each configuration
avg_ndcg_scores = {config: np.mean(scores) for config, scores in ndcg_scores.items()}
print("Average NDCG@5 for each configuration:", avg_ndcg_scores)


<bound method Elasticsearch.search of <Elasticsearch(['http://localhost:9200'])>>